In [33]:
import pandas as pd
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

import statsmodels.api as sm

In [2]:
df1 = pd.read_csv('../data/region/就业总人数.csv')
df2 = pd.read_csv('../data/region/高学历比例.csv')

In [3]:
df1 = pd.melt(df1, id_vars=['地区'], var_name='年份', value_name='总人数')
df2 = pd.melt(df2, id_vars=['地区'], var_name='年份', value_name='高学历人数占比')

In [12]:
df1.head()

,地区,年份,总人数
0,北京,1999,618.58
1,天津,1999,508.14
2,河北,1999,3322.30
3,山西,1999,1402.19
4,内蒙古,1999,1056.70


In [13]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 744 entries, 0 to 743
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   地区      744 non-null    object 
 1   年份      744 non-null    object 
 2   总人数     744 non-null    float64
dtypes: float64(1), object(2)
memory usage: 17.6+ KB


In [5]:
df2.head()

,地区,年份,高学历人数占比
0,北京,2021,43.72
1,天津,2021,28.13
2,河北,2021,8.61
3,山西,2021,11.29
4,内蒙古,2021,13.56


In [40]:
human_results = {}
human_models = {}
human_sm_res = {}
human_predict = {}

In [41]:
for region in df1['地区'].unique():
    df_region = df1[df1['地区'] == region]

    train_df = df_region[df_region['年份'] != '2022']
    test_df = df_region[df_region['年份'] == '2022']

    poly = PolynomialFeatures(degree=3)
    
    X_train = poly.fit_transform(train_df[['年份']])
    X_test = poly.transform(test_df[['年份']])
    
    model = LinearRegression()
    
    y_train = train_df['总人数']
    model.fit(X_train, y_train)
    
    y_test = test_df['总人数']
    y_pred = model.predict(X_test)
    
    error = mean_squared_error(y_test, y_pred)
    human_results[region] = error
    
    X_train_with_intercept = sm.add_constant(X_train)
    model_with_intercept = sm.OLS(y_train, X_train_with_intercept)
    results_with_intercept = model_with_intercept.fit()
    human_sm_res[region] = results_with_intercept
    
    poly = PolynomialFeatures(degree=3)
    train_df = df_region
    X_train = poly.fit_transform(train_df[['年份']])
    model = LinearRegression()
    y_train = train_df['总人数']
    model.fit(X_train, y_train)

    human_models[region] = model
    
    year_to_predict = 2023
    X_predict = pd.DataFrame([[year_to_predict]], columns=['年份'])
    X_predict = poly.transform(X_predict)

    human_predict[region] = model.predict(X_predict)

In [42]:
for region, error in human_results.items():
    print(f'{region} 的误差为 {error}')

北京 的误差为 142.19030201706266
天津 的误差为 2895.177641041883
河北 的误差为 48857.235083069885
山西 的误差为 7094.3998697398965
内蒙古 的误差为 16581.043493741923
辽宁 的误差为 15748.490423711672
吉林 的误差为 9921.79589667574
黑龙江 的误差为 51100.19528949251
上海 的误差为 794.555814966734
江苏 的误差为 5423.370688892248
浙江 的误差为 13683.046409533572
安徽 的误差为 48782.76021960855
福建 的误差为 2521.7352847315988
江西 的误差为 30.46301412007699
山东 的误差为 151.44971529171767
河南 的误差为 43774.649443253555
湖北 的误差为 5098.673305894496
湖南 的误差为 58721.944456964615
广东 的误差为 27431.685483473997
广西 的误差为 3309.8698004064972
海南 的误差为 53.18562573405586
重庆 的误差为 970.8025108553741
四川 的误差为 7590.464617762881
贵州 的误差为 37270.55331358351
云南 的误差为 4854.697714691577
西藏 的误差为 0.05573171294814307
陕西 的误差为 129.3810348276162
甘肃 的误差为 942.4067569970384
青海 的误差为 178.68089998939257
宁夏 的误差为 8.954280591959147
新疆 的误差为 432.1498300782732


In [43]:
for region, result in human_sm_res.items():
    print(region+":")
    print(result.summary())

北京:
                            OLS Regression Results                            
Dep. Variable:                    总人数   R-squared:                       0.982
Model:                            OLS   Adj. R-squared:                  0.980
Method:                 Least Squares   F-statistic:                     531.9
Date:                Sun, 05 Nov 2023   Prob (F-statistic):           4.58e-18
Time:                        20:29:13   Log-Likelihood:                -108.88
No. Observations:                  23   AIC:                             223.8
Df Residuals:                      20   BIC:                             227.2
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -4.8057      0.470    -10.215     

In [45]:
for region, result in human_predict.items():
    print(f'{region} 在 2023 年的预测人口值为 {result[0]}')

北京 在 2023 年的预测人口值为 1092.0027374625206
天津 在 2023 年的预测人口值为 550.1916310787201
河北 在 2023 年的预测人口值为 3321.2553160190582
山西 在 2023 年的预测人口值为 1542.7703018188477
内蒙古 在 2023 年的预测人口值为 1025.848415851593
辽宁 在 2023 年的预测人口值为 1941.863510608673
吉林 在 2023 年的预测人口值为 1048.4356734752655
黑龙江 在 2023 年的预测人口值为 1066.5463497638702
上海 在 2023 年的预测人口值为 1285.6560816764832
江苏 在 2023 年的预测人口值为 4821.252161085606
浙江 在 2023 年的预测人口值为 4026.7859119176865
安徽 在 2023 年的预测人口值为 2636.7150135040283
福建 在 2023 年的预测人口值为 2098.633153438568
江西 在 2023 年的预测人口值为 2154.382435441017
山东 在 2023 年的预测人口值为 5227.510485082865
河南 在 2023 年的预测人口值为 4945.574973106384
湖北 在 2023 年的预测人口值为 3277.9985803365707
湖南 在 2023 年的预测人口值为 2864.1388835906982
广东 在 2023 年的预测人口值为 6943.9419577121735
广西 在 2023 年的预测人口值为 2448.840286254883
海南 在 2023 年的预测人口值为 527.6762027144432
重庆 在 2023 年的预测人口值为 1586.9566669464111
四川 在 2023 年的预测人口值为 4783.531584382057
贵州 在 2023 年的预测人口值为 2047.5849866867065
云南 在 2023 年的预测人口值为 2627.200053215027
西藏 在 2023 年的预测人口值为 191.7249630689621
陕西 在 2023 年的预测人口值为 2054

In [46]:
human_predict_res = []

In [47]:
for region, result in human_predict.items():
    human_predict_res.append([region, result[0]])

In [48]:
human_predict_res = pd.DataFrame(human_predict_res, columns=['地区', '总人数'])

In [49]:
human_predict_res.to_csv('../data/region/就业总人数预测2023.csv', index=False)

In [60]:
structure_results = {}
structure_models = {}
structure_sm_res = {}
structure_predict = {}

In [61]:
for region in df2['地区'].unique():
    df_region = df2[df2['地区'] == region]

    train_df = df_region[df_region['年份'] != '2021']
    test_df = df_region[df_region['年份'] == '2021']

    poly = PolynomialFeatures(degree=3)
    
    X_train = poly.fit_transform(train_df[['年份']])
    X_test = poly.transform(test_df[['年份']])
    
    model = LinearRegression()
    
    y_train = train_df['高学历人数占比']
    model.fit(X_train, y_train)
    
    y_test = test_df['高学历人数占比']
    y_pred = model.predict(X_test)
    
    error = mean_squared_error(y_test, y_pred)
    structure_results[region] = error
    
    X_train_with_intercept = sm.add_constant(X_train)
    model_with_intercept = sm.OLS(y_train, X_train_with_intercept)
    results_with_intercept = model_with_intercept.fit()
    structure_sm_res[region] = results_with_intercept
    
    poly = PolynomialFeatures(degree=3)
    train_df = df_region
    X_train = poly.fit_transform(train_df[['年份']])
    model = LinearRegression()
    y_train = train_df['高学历人数占比']
    model.fit(X_train, y_train)

    structure_models[region] = model
    
    year_to_predict = 2023
    X_predict = pd.DataFrame([[year_to_predict]], columns=['年份'])
    X_predict = poly.transform(X_predict)

    structure_predict[region] = model.predict(X_predict)

In [62]:
for region, error in structure_results.items():
    print(f'{region} 的误差为 {error}')

北京 的误差为 0.567653591631237
天津 的误差为 0.16982835006590985
河北 的误差为 0.399405382449379
山西 的误差为 1.41690373775414
内蒙古 的误差为 0.7314635902154505
辽宁 的误差为 0.023159115335572484
吉林 的误差为 1.144331412188071
黑龙江 的误差为 0.738655793730015
上海 的误差为 2.0052344284122663
江苏 的误差为 0.6707336105814593
浙江 的误差为 6.100525909418627
安徽 的误差为 0.16287313244318602
福建 的误差为 2.077226150787
江西 的误差为 0.022042831169087607
山东 的误差为 1.630470838044287
河南 的误差为 0.23294613186859586
湖北 的误差为 0.12172974757758252
湖南 的误差为 0.013209046122230653
广东 的误差为 0.011450968858173927
广西 的误差为 0.2244884072755435
海南 的误差为 1.5039739152423712
重庆 的误差为 0.028146187894171663
四川 的误差为 0.04894744489647277
贵州 的误差为 0.29584875634772373
云南 的误差为 1.068198865990328
西藏 的误差为 7.55703039766289
陕西 的误差为 0.252795544092118
甘肃 的误差为 2.817883908851908
青海 的误差为 0.01344112188402996
宁夏 的误差为 2.8666594592252226
新疆 的误差为 11.673395581329638


In [63]:
for region, result in structure_sm_res.items():
    print(region+":")
    print(result.summary())

北京:
                            OLS Regression Results                            
Dep. Variable:                高学历人数占比   R-squared:                       0.952
Model:                            OLS   Adj. R-squared:                  0.946
Method:                 Least Squares   F-statistic:                     167.4
Date:                Sun, 05 Nov 2023   Prob (F-statistic):           6.55e-12
Time:                        21:09:59   Log-Likelihood:                -45.478
No. Observations:                  20   AIC:                             96.96
Df Residuals:                      17   BIC:                             99.94
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.0538      0.058     -0.932     

In [64]:
for region, result in structure_predict.items():
    print(f'{region} 在 2023 年的预测高学历人数占比为 {result[0]}')

北京 在 2023 年的预测高学历人数占比为 44.732994236052036
天津 在 2023 年的预测高学历人数占比为 30.806589514017105
河北 在 2023 年的预测高学历人数占比为 9.53098376840353
山西 在 2023 年的预测高学历人数占比为 12.218328032642603
内蒙古 在 2023 年的预测高学历人数占比为 15.093787245452404
辽宁 在 2023 年的预测高学历人数占比为 16.704669499769807
吉林 在 2023 年的预测高学历人数占比为 12.543939612805843
黑龙江 在 2023 年的预测高学历人数占比为 12.543269934132695
上海 在 2023 年的预测高学历人数占比为 38.109335171058774
江苏 在 2023 年的预测高学历人数占比为 14.46650730445981
浙江 在 2023 年的预测高学历人数占比为 14.372412785887718
安徽 在 2023 年的预测高学历人数占比为 10.359351402148604
福建 在 2023 年的预测高学历人数占比为 11.98296308517456
江西 在 2023 年的预测高学历人数占比为 10.247252751141787
山东 在 2023 年的预测高学历人数占比为 9.363710571080446
河南 在 2023 年的预测高学历人数占比为 7.6647530272603035
湖北 在 2023 年的预测高学历人数占比为 11.197781916707754
湖南 在 2023 年的预测高学历人数占比为 10.219200866296887
广东 在 2023 年的预测高学历人数占比为 14.283345768693835
广西 在 2023 年的预测高学历人数占比为 8.890457903966308
海南 在 2023 年的预测高学历人数占比为 12.094431584700942
重庆 在 2023 年的预测高学历人数占比为 12.388460397720337
四川 在 2023 年的预测高学历人数占比为 9.668370543047786
贵州 在 2023 年的预测高学历人数占比为 9.05587510485202

In [65]:
structure_predict_res = []

In [66]:
for region, result in structure_predict.items():
    structure_predict_res.append([region, result[0]])

In [67]:
structure_predict_res = pd.DataFrame(structure_predict_res, columns=['地区', '高学历人数占比'])

In [68]:
structure_predict_res.to_csv('../data/region/高学历比例预测2023.csv', index=False)